# Building a Sentiment Classifier for Products Description from Amazon Reviews

The idea here is to elaborate on the idea of using a Sentiment Classifier Model from Amazon Products reviews in order to analyze and classify products descriptions of Natural Medicine products.

In [2]:
import graphlab
graphlab.canvas.set_target('ipynb')

In [14]:
# loads the dataset into a SFrame for futher analysis
reviews_data = graphlab.SFrame.read_json('Dumps/ratings_dump.json')

Finished parsing file C:\Users\diego\NaturalMedicineProject\Dumps\ratings_dump.json

Parsing completed. Parsed 1 lines in 0.307207 secs.

In [68]:
reviews_data = reviews_data.select_columns(['asin', 'reviewerName', 'helpful', 'overall', 'summary', 'reviewText'])
reviews_data.head()

asin,reviewerName,helpful,overall,summary,reviewText
159985130X,AnnN,"[1, 1]",5.0,Handy little gadget,This is a great littlegadget to have around. ...
159985130X,"AZ buyer ""AZ buyer""","[1, 1]",4.0,Small & may need toencourage battery ...,I would recommend thisfor a travel magnifier ...
159985130X,"Bob Tobias ""RobertTobias"" ...","[75, 77]",4.0,Very good but not great,What I liked was thequality of the lens and ...
159985130X,Cat lover,"[56, 60]",4.0,great addition to yourpurse ...,Love the Great pointlight pocket magnifier! ...
159985130X,Cricketoes,"[1, 1]",5.0,Very nice and convenient.,This is very nice. Youpull out on the magni ...
159985130X,Dwight,"[2, 3]",5.0,"$9.99, pretty and cute",The light comes on whenthe item is pulled. ...
159985130X,Eusebius,"[0, 0]",4.0,Lightweight and efficient,These are lightweight andefficient and have some ...
159985130X,"Gary ""Gary""","[2, 2]",5.0,Excellent Design andFunctionaality ...,We bought one for roadtrips and trying to ...
159985130X,guru,"[1, 1]",3.0,Okay,The screen of themagnifier is small. If ...
159985130X,Jeffrey E. Cornett,"[1, 1]",4.0,Pocket magnifier,This pocket magnifier isnice and compact. The ...


In [9]:
len(reviews_data)

10000

In [70]:
reviews_data['asin'].show()

In [72]:
reviews_data['overall'].show(view='Categorical')

## Building the Word Count Vector for both Summary and Review text Sentiment Model

In [69]:
def sum_dict(a, b):
    c = a.copy()
    c.update(b)
    return c

def ngram_it(column, size, data=[]):
    """ Builds a word count vector on ngrams of a given size for a given text column.
        
        column: the name of the column on the SFrame
        size: the size of the ngrams
        data: used on the recursion
        
        return: SArray with a dictionary of {ngram: count} for each text
    """
    if size == 0:
        res = []
        
        for idx in xrange(len(data[0])):
            a = {}
            for i in xrange(len(data)):
                a = sum_dict(a, data[i][idx])
            res.append(a)
        
        return graphlab.SArray(data=res)
        
    e = graphlab.text_analytics.count_ngrams(column, n = size, to_lower=True)
    data.append(e)
    return ngram_it(column, size - 1, data)

# Lets create a n=3 ngrams word-count vector for each review
reviews_data['summary_word_count'] = ngram_it(reviews_data['summary'], 3)
reviews_data['review_word_count'] = ngram_it(reviews_data['reviewText'], 3)

reviews_data.head()

asin,reviewerName,helpful,overall,summary,reviewText
159985130X,AnnN,"[1, 1]",5.0,Handy little gadget,This is a great littlegadget to have around. ...
159985130X,"AZ buyer ""AZ buyer""","[1, 1]",4.0,Small & may need toencourage battery ...,I would recommend thisfor a travel magnifier ...
159985130X,"Bob Tobias ""RobertTobias"" ...","[75, 77]",4.0,Very good but not great,What I liked was thequality of the lens and ...
159985130X,Cat lover,"[56, 60]",4.0,great addition to yourpurse ...,Love the Great pointlight pocket magnifier! ...
159985130X,Cricketoes,"[1, 1]",5.0,Very nice and convenient.,This is very nice. Youpull out on the magni ...
159985130X,Dwight,"[2, 3]",5.0,"$9.99, pretty and cute",The light comes on whenthe item is pulled. ...
159985130X,Eusebius,"[0, 0]",4.0,Lightweight and efficient,These are lightweight andefficient and have some ...
159985130X,"Gary ""Gary""","[2, 2]",5.0,Excellent Design andFunctionaality ...,We bought one for roadtrips and trying to ...
159985130X,guru,"[1, 1]",3.0,Okay,The screen of themagnifier is small. If ...
159985130X,Jeffrey E. Cornett,"[1, 1]",4.0,Pocket magnifier,This pocket magnifier isnice and compact. The ...


## Building a Sentiment Classifier

First, lest filter the reviews in order to focus only on the relevant ones.

In [75]:
relevant_reviews = reviews_data[reviews_data['overall'] != 3]
relevant_reviews['overall'].show(view='Categorical')

In [78]:
relevant_reviews['sentiment'] = relevant_reviews['overall'] >= 4
relevant_reviews['sentiment'].show(view='Categorical')

### Training the sentiment classifier

In [80]:
training_data, testing_data = relevant_reviews.random_split(.8, seed = 0)

Lets create a sentiment classifier for the summary word vector

In [84]:
summary_sentiment_model = graphlab.logistic_classifier.create(training_data,
                                                              target='sentiment',
                                                              features=['summary_word_count'],
                                                              max_iterations=50,
                                                              validation_set=testing_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 7404

Number of classes           : 2

...

| 11        | 16       | 1.000000  | 0.438293     | 0.997029          | 0.924016            |

| 50        | 67       | 0.500000  | 1.874252     | 0.997569          | 0.920133            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Now, for the review text

In [87]:
review_sentiment_model = graphlab.logistic_classifier.create(training_data,
                                                              target='sentiment',
                                                              features=['review_word_count'],
                                                              max_iterations=10,
                                                              validation_set=testing_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 7404

Number of classes           : 2

...

| 6         | 11       | 1.000000  | 2.467646     | 1.000000          | 0.895175            |

| 10        | 15       | 1.000000  | 3.838565     | 1.000000          | 0.895175            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

## Lets evaluate the models using the ROC Curve metric

In [88]:
summary_sentiment_model.evaluate(testing_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+------+-----+
 | threshold |      fpr       |      tpr       |  p   |  n  |
 +-----------+----------------+----------------+------+-----+
 |    0.0    |      1.0       |      1.0       | 1613 | 190 |
 |   1e-05   | 0.836842105263 | 0.997520148791 | 1613 | 190 |
 |   2e-05   | 0.805263157895 | 0.997520148791 | 1613 | 190 |
 |   3e-05   | 0.794736842105 | 0.997520148791 | 1613 | 190 |
 |   4e-05   | 0.789473684211 | 0.996900185989 | 1613 | 190 |
 |   5e-05   | 0.778947368421 | 0.996900185989 | 1613 | 190 |
 |   6e-05   | 0.773684210526 | 0.996900185989 | 1613 | 190 |
 |   7e-05   | 0.773684210526 | 0.996280223187 | 1613 | 190 |
 |   8e-05   | 0.773684210526 | 0.996280223187 | 1613 | 190 |
 |   9e-05   | 0.768421052632 | 0.996280223187 | 1613 | 190 |
 +-----------+----------------+----------------+------+-----+
 [100001 rows x 5 columns]

In [89]:
review_sentiment_model.evaluate(testing_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+-----+------+-----+
 | threshold |      fpr       | tpr |  p   |  n  |
 +-----------+----------------+-----+------+-----+
 |    0.0    |      1.0       | 1.0 | 1613 | 190 |
 |   1e-05   | 0.994736842105 | 1.0 | 1613 | 190 |
 |   2e-05   | 0.994736842105 | 1.0 | 1613 | 190 |
 |   3e-05   | 0.994736842105 | 1.0 | 1613 | 190 |
 |   4e-05   | 0.994736842105 | 1.0 | 1613 | 190 |
 |   5e-05   | 0.994736842105 | 1.0 | 1613 | 190 |
 |   6e-05   | 0.994736842105 | 1.0 | 1613 | 190 |
 |   7e-05   | 0.994736842105 | 1.0 | 1613 | 190 |
 |   8e-05   | 0.994736842105 | 1.0 | 1613 | 190 |
 |   9e-05   | 0.994736842105 | 1.0 | 1613 | 190 |
 +-----------+----------------+-----+------+-----+
 [100001 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [90]:
summary_sentiment_model.show(view='Evaluation')

In [91]:
review_sentiment_model.show(view='Evaluation')

## Applying the model on some specific product's review dataset

### B000053L6Y: Clubman Pinaud After Shave Lotion

<img src="http://images.cdn0.buscalibre.com/531368dd6927b8c632000004.__grande__.jpg">

In [100]:
shave_lotion_reviews = reviews_data[reviews_data['asin'] == 'B000053L6Y']
shave_lotion_reviews.select_columns(['reviewerName','summary', 'reviewText', 'overall', 'helpful']).head()

reviewerName,summary,reviewText,overall,helpful
A. Booker,Barbershop memories,Old school barbershopsmell. Comes on strong ...,4.0,"[0, 0]"
AJC,Classis Aftershave.,Been using this sameaftershave for over 40 ...,5.0,"[1, 1]"
ajk170,Limited shipping,Smells like the barbershop from when I was a ...,5.0,"[1, 1]"
"Al Errico ""Big Al""",Smell like you just wereat the barber ...,This is an oldtime scentthat is clean and ...,5.0,"[1, 1]"
Allen Coates,I purchased this based ongood reviews on Amazon ...,I am used to a lotionafter my shave and have ...,3.0,"[1, 1]"
Amar Patel,Not a Lotion!,I bought this thinking itwas an actual lotion but ...,4.0,"[1, 1]"
"Amazon Customer""Gearhed79"" ...",Not for me.,It's a cheap aftershavethat smells like orig ...,2.0,"[1, 3]"
Amazon Customer,Should have discoveredthis years ago... ...,I used to see Clubman atthe drugstore (in glass ...,5.0,"[0, 0]"
Amazon Customer,Welcome back to yourchildhood barber's ...,"I just rediscoveredPinaud Clubman, and was ...",5.0,"[0, 1]"
"Amazon Customer ""tde""",nice smell,Actually it has a nicesmell to it . Mascul ...,3.0,"[0, 1]"


In [137]:
shave_lotion_reviews['overall'].show(view='Categorical')

In [101]:
shave_lotion_reviews['predicted_sentiment_summary'] = summary_sentiment_model.predict(shave_lotion_reviews, 
                                                                                      output_type='probability')
shave_lotion_reviews['predicted_sentiment_review'] = review_sentiment_model.predict(shave_lotion_reviews, 
                                                                                      output_type='probability')

In [103]:
shave_lotion_reviews.select_columns(['summary', 'reviewText', 'overall', 
                                     'predicted_sentiment_summary',
                                     'predicted_sentiment_review']).head()

summary,reviewText,overall,predicted_sentiment_summary ...,predicted_sentiment_review ...
Barbershop memories,Old school barbershopsmell. Comes on strong ...,4.0,0.999999802087,0.999975902357
Classis Aftershave.,Been using this sameaftershave for over 40 ...,5.0,0.999999917959,0.99999750534
Limited shipping,Smells like the barbershop from when I was a ...,5.0,0.999999711058,1.0
Smell like you just wereat the barber ...,This is an oldtime scentthat is clean and ...,5.0,0.997301141934,0.999986641948
I purchased this based ongood reviews on Amazon ...,I am used to a lotionafter my shave and have ...,3.0,0.999999238343,0.996475805611
Not a Lotion!,I bought this thinking itwas an actual lotion but ...,4.0,0.813860293311,0.992873455675
Not for me.,It's a cheap aftershavethat smells like orig ...,2.0,0.119305391795,7.8314939723e-06
Should have discoveredthis years ago... ...,I used to see Clubman atthe drugstore (in glass ...,5.0,0.999998744249,0.999999999999
Welcome back to yourchildhood barber's ...,"I just rediscoveredPinaud Clubman, and was ...",5.0,0.999999978021,1.0
nice smell,Actually it has a nicesmell to it . Mascul ...,3.0,0.953509241977,0.989354622579


In [105]:
shave_lotion_reviews_sorted_summary = shave_lotion_reviews.sort('predicted_sentiment_summary', ascending=False)
shave_lotion_reviews_sorted_review = shave_lotion_reviews.sort('predicted_sentiment_review', ascending=False)

In [128]:
def print_review(review):
    print 'Summary: {0}\n'.format(review['summary'])
    print 'Rating: {0}'.format(review['overall'])
    print 'Helpfulness: {0}/{1}'.format(review['helpful'][1], review['helpful'][0])
    print 'Sentiment from the Summary model: {0}'.format(review['predicted_sentiment_summary'])
    print 'Sentiment from the Review model: {0}'.format(review['predicted_sentiment_review'])
    print '\n'
    print review['reviewText']
    print '\n'

In [130]:
print 'Best reviews:\n'
print_review(shave_lotion_reviews_sorted_review[0])
print_review(shave_lotion_reviews_sorted_review[10])
print_review(shave_lotion_reviews_sorted_review[20])

print 'Normal reviews:\n'
l = len(shave_lotion_reviews_sorted_review)
print_review(shave_lotion_reviews_sorted_review[l / 2 - 10])
print_review(shave_lotion_reviews_sorted_review[l / 2])
print_review(shave_lotion_reviews_sorted_review[l / 2 + 10])

print 'Worst reviews:\n'
print_review(shave_lotion_reviews_sorted_review[-1])
print_review(shave_lotion_reviews_sorted_review[-20])
print_review(shave_lotion_reviews_sorted_review[-30])

Best reviews:

Summary: Give it a Go, your face will Glow.

Rating: 4.0
Helpfulness: 1/1
Sentiment from the Summary model: 0.999999895557
Sentiment from the Review model: 1.0


I have not used after shave in 20 years, so I am a born again virgin on this product type.The Clubman After Shave is pleasant in smell, but I must watch how much I use - can be overbearing if you are not careful.  Reminds me of my grandpa, when we went visiting, to me, thats a good thing.  Smells nice, rolls on easy.I do notice that my post shave routine is soothed by using aftershave to ease the nicks and such.  I have recently switched over to Double Edge safety razor on the toss away multiple head shavers.  The DE Safety razors are very sharp, and nicks now occur, were they were seldom on the duller toss aways.  So a good after shaved seemed a worthy product to try, and it works.  I guess granpa was pretty smart after all.Why 4 stars, not 5 - I only 5 for truly outstanding products.  This is very good, and re

### B0000CEO82: Gluten Free Bar - Lemon Zest - (1.69 Ounce Snack Bar, 15 Count)

<img src="https://images-na.ssl-images-amazon.com/images/G/01/aplus/detail-page/c26-B0000CEO82-1-s.jpg" />

In [132]:
gluten_free_bar_reviews = reviews_data[reviews_data['asin'] == 'B0000CEO82']
gluten_free_bar_reviews.select_columns(['reviewerName','summary', 'reviewText', 'overall', 'helpful']).head()

reviewerName,summary,reviewText,overall,helpful
A book lover in AzleTexas ...,Just love my Luna bars!,I have turned severalpeople onto these. They ...,5.0,"[0, 0]"
Alison Link,Good,Definitely a good snack &healthy if you are ...,4.0,"[0, 0]"
"Almonds1 ""Almond1""",No men allowed?,This item is labeled asnutrition for women and ...,4.0,"[0, 0]"
Amazon Customer,awesome,Love this Luna Barflavor! Been a Luna Bar ...,5.0,"[0, 0]"
Amazon Customer,Scrumptious!!,The Luna ChocolatePeppermint Stick Bars ...,5.0,"[0, 0]"
"Amazon Customer ""MichelleKilpatrick"" ...",Great Tasting!,These bars taste great.They also leave me full. ...,4.0,"[0, 0]"
AmazonReviewer,Delicious,"These are delicious and agreat snack, good before ...",5.0,"[0, 0]"
"Amy ""Amy""",One of my favoritenutrition bars ...,These Luna lemon bars areone of my go-to snacks. ...,5.0,"[0, 0]"
animalmom,Love 'em!!!,This is my favoriteflavor and they are hard ...,5.0,"[0, 0]"
"Ann ""one-eye""",luna bar gets better,Luna has cut the fatcontent and increased ...,4.0,"[0, 0]"


In [138]:
gluten_free_bar_reviews['overall'].show(view='Categorical')

In [133]:
gluten_free_bar_reviews['predicted_sentiment_summary'] = summary_sentiment_model.predict(gluten_free_bar_reviews, 
                                                                                      output_type='probability')
gluten_free_bar_reviews['predicted_sentiment_review'] = review_sentiment_model.predict(gluten_free_bar_reviews, 
                                                                                      output_type='probability')

In [134]:
gluten_free_bar_reviews.select_columns(['summary', 'reviewText', 'overall', 
                                     'predicted_sentiment_summary',
                                     'predicted_sentiment_review']).head()

summary,reviewText,overall,predicted_sentiment_summary ...,predicted_sentiment_review ...
Just love my Luna bars!,I have turned severalpeople onto these. They ...,5.0,0.999999999708,1.0
Good,Definitely a good snack &healthy if you are ...,4.0,0.999616896264,0.999948125353
No men allowed?,This item is labeled asnutrition for women and ...,4.0,0.999999748748,0.99997128994
awesome,Love this Luna Barflavor! Been a Luna Bar ...,5.0,0.999991466247,1.0
Scrumptious!!,The Luna ChocolatePeppermint Stick Bars ...,5.0,0.999999753235,0.999993256255
Great Tasting!,These bars taste great.They also leave me full. ...,4.0,0.999997888532,0.999963530568
Delicious,"These are delicious and agreat snack, good before ...",5.0,0.999993589131,0.999967949479
One of my favoritenutrition bars ...,These Luna lemon bars areone of my go-to snacks. ...,5.0,0.999999981382,0.999997813811
Love 'em!!!,This is my favoriteflavor and they are hard ...,5.0,0.999999832218,0.999983883915
luna bar gets better,Luna has cut the fatcontent and increased ...,4.0,0.999999683135,0.999999187361


In [135]:
gluten_free_bar_reviews_sorted_summary = gluten_free_bar_reviews.sort('predicted_sentiment_summary', ascending=False)
gluten_free_bar_reviews_sorted_review = gluten_free_bar_reviews.sort('predicted_sentiment_review', ascending=False)

In [136]:
print 'Best reviews:\n'
print_review(gluten_free_bar_reviews_sorted_review[0])
print_review(gluten_free_bar_reviews_sorted_review[10])
print_review(gluten_free_bar_reviews_sorted_review[20])

print 'Normal reviews:\n'
l = len(gluten_free_bar_reviews_sorted_review)
print_review(gluten_free_bar_reviews_sorted_review[l / 2 - 10])
print_review(gluten_free_bar_reviews_sorted_review[l / 2])
print_review(gluten_free_bar_reviews_sorted_review[l / 2 + 10])

print 'Worst reviews:\n'
print_review(gluten_free_bar_reviews_sorted_review[-1])
print_review(gluten_free_bar_reviews_sorted_review[-20])
print_review(gluten_free_bar_reviews_sorted_review[-30])

Best reviews:

Summary: awesome

Rating: 5.0
Helpfulness: 0/0
Sentiment from the Summary model: 0.999991466247
Sentiment from the Review model: 1.0


Love this Luna Bar flavor! Been a Luna Bar fan for the last eight years or so. When I was a teenager, I struggled with anorexia nervosa, but I ate this as bar as of the few low-calorie things I would eat ... and I think the high nutrient content of this bar helped me stay healthy enough to make it this far, all things considered. So if you're a parent worried about your teen not eating, in addition to getting them psychiatric help, consider introducing them to these. The nutrition content is not quite so terrifying to people like me who suffered an eating disorder, but it's got a lot of good things in it to keep their health from deteriorating quite as quickly (at least it's better than other cereal bars that are nothing but sugar). Nowadays, I just like this as a snack to keep my energy up while studying or on-the-go running errands. Tas

## Creating the table n-grams for the new Sentiment Classifier

#### F(@frequency, @relevance, @accuracy) = @quality

### Calculating the @frequency of each n-gram on the whole dataset

#### In order to calculate the frequency on each review's text we will calculate the TF-IDF of each ngram in relation to the entire corpus

In [146]:
reviews_data['tf_idf'] = graphlab.text_analytics.tf_idf(reviews_data['review_word_count'])

In [149]:
reviews_data[['reviewText', 'review_word_count', 'tf_idf']].head()

reviewText,review_word_count,tf_idf
This is a great littlegadget to have around. ...,"{'d bought one': 1L, 'tohave': 1L, 'one with': ...","{'d bought one':9.210340371976184, 'to ..."
I would recommend thisfor a travel magnifier ...,"{'box but after': 1L,'all': 1L, 'travel ...","{'box but after':9.210340371976184, 'a ..."
What I liked was thequality of the lens and ...,"{'reviewers': 1L, 'betterquality': 1L, 'this ...","{'reviewers':4.625372893305611, ..."
Love the Great pointlight pocket magnifier! ...,"{'if you': 1L, 'forget':1L, 'case to protec': ...","{'if you':2.017406150760383, ..."
This is very nice. Youpull out on the magni ...,"{'on the magnifier': 1L,'magnifier when you': ...","{'on the magnifier':9.210340371976184, ..."
The light comes on whenthe item is pulled. ...,"{'cute': 1L, 'to have':1L, 'magnifier and the': ...","{'cute':6.907755278982137, 'to ..."
These are lightweight andefficient and have some ...,"{'all': 1L, 'not take a':1L, 's easy': 1L, ...","{'all':1.6586281566248722, 'not ..."
We bought one for roadtrips and trying to ...,"{'if you': 1L, 'feel icouldn': 1L, 'where the ...","{'if you':2.017406150760383, 'feel ..."
The screen of themagnifier is small. If ...,"{'screw a': 1L, 'text':1L, 'would have to': 1L, ...","{'screw a':9.210340371976184, ..."
This pocket magnifier isnice and compact. The ...,"{'and': 2L, 'storing':1L, 'slide out': 1L, ...","{'and':0.48541741361809954, ..."


In [160]:
ngram_frequency_table = reviews_data[['tf_idf']].stack('tf_idf', new_column_name=['ngram', 'frequency'])

In [188]:
ngram_frequency_table = ngram_frequency_table.sort('frequency', ascending=False).unique()

In [189]:
ngram_frequency_table.print_rows(num_rows=50)

+---------------+-------------------------+
|   frequency   |          ngram          |
+---------------+-------------------------+
| 9.21034037198 |        mouth try        |
| 9.21034037198 |     default for your    |
| 9.21034037198 |        top shines       |
| 9.21034037198 |       to and stop       |
| 9.21034037198 |    mossy growth built   |
| 9.21034037198 |     for generics and    |
| 9.21034037198 |     "big name" brand    |
| 9.21034037198 |    actually followed    |
| 9.21034037198 |       drool can t       |
| 7.01311579464 |     ear thermometer     |
| 9.21034037198 |       in michigan       |
| 9.21034037198 |      extremely soft     |
| 8.51719319142 |      pulling it out     |
| 6.90775527898 |       your stomach      |
| 9.21034037198 |        my temp at       |
| 9.21034037198 |   a reusable container  |
| 9.21034037198 |     my recorder were    |
| 9.21034037198 |      be using harsh     |
| 9.21034037198 |      another dosage     |
| 9.21034037198 |     onto your 